Crearemos los dataframes utilizados en las funciones


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Steam_games_limpio.csv to Steam_games_limpio.csv
Saving User_items_limpio.csv to User_items_limpio.csv
Saving User_review_limpio.csv to User_review_limpio.csv


In [ ]:
df_reviews = pd.read_csv("User_review_limpio.csv", encoding="utf-8")
df_games = pd.read_csv("Steam_games_limpio.csv", encoding="utf-8")
df_items = pd.read_csv("User_items_limpio.csv", encoding="utf-8")

In [ ]:
df_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'sentiment_analysis', 'year_posted'], dtype='object')

In [ ]:
df_games.columns

Index(['app_name', 'title', 'id', 'developer', 'price_number', 'release_year',
       'main_genre', 'igualdad'],
      dtype='object')

In [ ]:
df_items.columns

Index(['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever'], dtype='object')

In [ ]:
df_items["playtime_hour"]= df_items["playtime_forever"]/60
df_items

,user_id,items_count,item_id,item_name,playtime_forever,playtime_hour
0,76561197970982479,277,10.0,Counter-Strike,6.0,0.100000
1,76561197970982479,277,20.0,Team Fortress Classic,0.0,0.000000
2,76561197970982479,277,30.0,Day of Defeat,7.0,0.116667
3,76561197970982479,277,40.0,Deathmatch Classic,0.0,0.000000
4,76561197970982479,277,50.0,Half-Life: Opposing Force,0.0,0.000000
...,...,...,...,...,...,...
5170010,76561198329548331,7,373330.0,All Is Dust,0.0,0.000000
5170011,76561198329548331,7,388490.0,One Way To Die: Steam Edition,3.0,0.050000
5170012,76561198329548331,7,521570.0,You Have 10 Seconds 2,4.0,0.066667
5170013,76561198329548331,7,519140.0,Minds Eyes,3.0,0.050000


## PlayTimeGenre
*Debe devolver año con mas horas jugadas para dicho género.*

In [ ]:
#Son necesarias main_genre, id y Release_year de df_games y la columna playtime_forever e item_id de df_items#
df_games2 = df_games[["main_genre", "release_year","id"]]
df_items2 = df_items[["playtime_hour", "item_id"]]
df_PlayTimeGenre_merge = pd.merge(df_items2, df_games2, left_on='item_id', right_on="id")
df_PlayTimeGenre_hour = df_PlayTimeGenre_merge.groupby(["main_genre", "release_year"])["playtime_hour"].sum().reset_index()
df_PlayTimeGenre_hour

,main_genre,release_year,playtime_hour
0,Action,1983,59.700000
1,Action,1984,6.400000
2,Action,1988,270.716667
3,Action,1989,10.116667
4,Action,1990,313.116667
...,...,...,...
263,Utilities,2016,1305.183333
264,Video Production,2015,5903.483333
265,Web Publishing,2005,1564.116667
266,Web Publishing,2015,3.100000


Guardamos el nuevo DF

In [ ]:
df_PlayTimeGenre_hour.to_csv('PlayTimeGenre.csv', index=False)

files.download('PlayTimeGenre.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_PlayTimeGenre_hour.to_parquet('PlayTimeGenre.parquet', index=False)

files.download('PlayTimeGenre.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## UserForGenre
Usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [ ]:
data_reviews2 = df_reviews[["item_id","user_id"]]

In [ ]:
data_items2 = df_items[["user_id","item_id","playtime_hour"]]

Unificamos las columnas


In [ ]:
merged_data = data_items2.merge(data_reviews2, on=["user_id", "item_id"], how="inner")

In [ ]:
#Ahora sumamos las columnas necesarias de games "item_id", "id", "main_genre"#
final_merged_data = merged_data.merge(df_games[["id","main_genre","release_year"]], left_on="item_id", right_on="id", how="inner")

Agrupamos ahora de acuerdo al usuario, el género del juego y el año

In [ ]:
df_UsersForGenre = final_merged_data.groupby(["user_id","main_genre","release_year"])["playtime_hour"].sum().reset_index()

In [ ]:
df_UsersForGenre

,user_id,main_genre,release_year,playtime_hour
0,--000--,Action,2009,49.150000
1,--ace--,Action,2011,21.150000
2,--ionex--,Action,2011,14.033333
3,--ionex--,Action,2012,591.300000
4,-2SV-vuLB-Kg,Action,2012,501.150000
...,...,...,...,...
34942,zyr0n1c,Action,2011,10.216667
34943,zyr0n1c,Action,2012,959.083333
34944,zyr0n1c,Action,2013,153.200000
34945,zyr0n1c,Action,2014,10.716667


In [ ]:
data_reviews3 = df_reviews[["item_id","user_id","year_posted"]]
data_items3 = df_items[["user_id","item_id","playtime_hour"]]
merged_data3 = data_items3.merge(data_reviews3, on=["user_id", "item_id"], how="inner")
final_merged_data3 = merged_data3.merge(df_games[["id","main_genre"]], left_on="item_id", right_on="id", how="inner")
df_UsersForGenre3 = final_merged_data3.groupby(["user_id","main_genre","year_posted"])["playtime_hour"].sum().reset_index()
df_UsersForGenre3

,user_id,main_genre,year_posted,playtime_hour
0,--000--,Action,2014,49.150000
1,--ace--,Action,2014,21.150000
2,--ionex--,Action,2013,591.300000
3,--ionex--,Action,2015,14.033333
4,-2SV-vuLB-Kg,Action,2014,515.916667
...,...,...,...,...
29759,zynxgameth,Action,2013,64.100000
29760,zyr0n1c,Action,2013,40.966667
29761,zyr0n1c,Action,2014,10.716667
29762,zyr0n1c,Action,2015,1112.283333


In [ ]:
df_UsersForGenre3.to_csv('UsersForGenre3.csv', index=False)

files.download('UsersForGenre3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_UsersForGenre3.to_parquet('UsersForGenre3.parquet', index=False)

files.download('UsersForGenre3.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


##UsersRecommend

In [ ]:
df_games4 = df_games[["id", "app_name"]].drop_duplicates()
df_reviews4 = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]
df_reviews_merged_ = pd.merge(df_reviews4, df_games4, left_on="item_id", right_on="id")
positive_reviews_ = df_reviews_merged_[(df_reviews_merged_["recommend"]== True) & (df_reviews_merged_["sentiment_analysis"] >=1)]
grouped_reviews_ = positive_reviews_.groupby(["year_posted", "item_id", "app_name"]).size().reset_index(name="recommendation_count")
df_UsersRecommend = (
    grouped_reviews_.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)
df_UsersRecommend

,year_posted,item_id,app_name,recommendation_count
0,2010,440,Team Fortress 2,10
1,2010,1250,Killing Floor,6
2,2010,630,Alien Swarm,4
3,2011,440,Team Fortress 2,76
4,2011,620,Portal 2,26
5,2011,105600,Terraria,23
6,2012,440,Team Fortress 2,249
7,2012,105600,Terraria,38
8,2012,4000,Garry's Mod,34
9,2013,440,Team Fortress 2,999


In [ ]:
df_UsersRecommend.to_csv('df_UsersRecommend.csv', index=False)

files.download('df_UsersRecommend.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_UsersRecommend.to_parquet('df_UsersRecommend.parquet', index=False)

files.download('df_UsersRecommend.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## UsersWorstDeveloper

In [ ]:
df_games4 = df_games[["id", "developer"]].drop_duplicates()
df_reviews4 = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]
df_reviews_merged4 = pd.merge(df_reviews4, df_games4, left_on="item_id", right_on="id")
negative_reviews4 = df_reviews_merged4[(df_reviews_merged4["recommend"]== False) & (df_reviews_merged4["sentiment_analysis"] == 0)]
grouped_reviews4 = negative_reviews4.groupby(["year_posted", "item_id", "developer"]).size().reset_index(name="recommendation_count")
df_UsersWorstDeveloper = (
    grouped_reviews4.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)
df_UsersWorstDeveloper

,year_posted,item_id,developer,recommendation_count
0,2011,18700,Broken Rules,1
1,2011,63940,1C Company,1
2,2012,440,Valve,1
3,2012,42920,NeoCoreGames,1
4,2012,55110,Volition,1
5,2013,730,Valve,25
6,2013,275850,Hello Games,20
7,2013,221100,Bohemia Interactive,13
8,2014,730,Valve,15
9,2014,221100,Bohemia Interactive,15


In [ ]:
df_UsersWorstDeveloper_final = df_UsersWorstDeveloper.groupby(["year_posted","developer"], as_index=False)["recommendation_count"].sum()
df_UsersWorstDeveloper_final

,year_posted,developer,recommendation_count
0,2011,1C Company,1
1,2011,Broken Rules,1
2,2012,NeoCoreGames,1
3,2012,Valve,1
4,2012,Volition,1
5,2013,Bohemia Interactive,13
6,2013,Hello Games,20
7,2013,Valve,25
8,2014,Bohemia Interactive,15
9,2014,Smartly Dressed Games,14


In [ ]:
df_UsersWorstDeveloper_final.to_csv('df_UsersWorstDeveloper_final.csv', index=False)

files.download('df_UsersWorstDeveloper_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_UsersWorstDeveloper_final.to_parquet('df_UsersWorstDeveloper_final.parquet', index=False)

files.download('df_UsersWorstDeveloper_final.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Sentiment_analysis

In [ ]:
df_reviews0 = df_reviews[["user_id", "item_id","recommend","year_posted","sentiment_analysis"]]
df_reviews1 = pd.merge(df_reviews0, df_games[["id","app_name","developer"]], left_on="item_id", right_on="id")
df_reviews02 = df_reviews1[["user_id","app_name","developer","recommend","year_posted","sentiment_analysis"]]
df_Sentiment_analysis = df_reviews02.groupby(["developer","sentiment_analysis"]).size().reset_index(name="sentiment_analysis_count")
df_Sentiment_analysis

,developer,sentiment_analysis,sentiment_analysis_count
0,07th Expansion,0,1
1,07th Expansion,1,2
2,"10th Art Studio,Adventure Productions",0,1
3,"10th Art Studio,Adventure Productions",2,1
4,10tons Ltd,2,1
...,...,...,...
3876,橘子班,0,1
3877,橘子班,1,2
3878,橘子班,2,1
3879,"高考恋爱委员会,橘子班",1,1


In [ ]:
df_Sentiment_analysis.to_csv('df_Sentiment_analysis.csv', index=False)

files.download('df_Sentiment_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_Sentiment_analysis.to_parquet('df_Sentiment_analysis.parquet', index=False)

files.download('df_Sentiment_analysis.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>